# Input/Output

In this section, we illustrate how to input and output **gridded** climate data with `cfr`.

`cfr` provides a useful class called `ClimateField` to handle **gridded** climate data.
It is essentially a wrapper of a `xarray.DataArray`, but with additional analysis and visualization functionalities added on.

Essentially, `cfr` supports below conversions:

- a netCDF file <=> `cfr.ClimateField`
- `xarray.DataArray` <=> `cfr.ClimateField`

Required data to complete this tutorial:

- GISTEMP surface temperature: [gistemp1200_GHCNv4_ERSSTv5.nc](https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz)
- HadCRUTv5 surface temperature: [HadCRUT.5.0.1.0.anomalies.ensemble_mean.nc](https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/non-infilled/HadCRUT.5.0.1.0.anomalies.ensemble_mean.nc)

In [21]:
%load_ext autoreload
%autoreload 2

import cfr
import pandas as pd
import numpy as np
import xarray as xr
import os
os.chdir('/glade/u/home/fengzhu/Github/cfr/docsrc/notebooks/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## a netCDF file => `cfr.ClimateField`

A simplest case is that the netCDF file contains only one variable with standard names for time, latitude, and longitude: `time`, `lat`, `lon`.
In this case, we just create a `cfr.ClimateField` object and call its `.load_nc()` method with a path to the netCDF file as the argument.

Sometimes, however, the netCDF file comes with multiple variables, in which case we need to specify the variable name via the `vn` argument:

In [23]:
dirpath = './data'
fd = cfr.ClimateField().load_nc(
    os.path.join(dirpath, 'gistemp1200_GHCNv4_ERSSTv5.nc'),  # path to the netCDF file
    vn='tempanomaly',  # specify the name of the variable to load
)
fd.da  # check the loaded `xarray.DataArray`

<xarray.DataArray 'tempanomaly' (time: 1718, lat: 90, lon: 180)>
[27831600 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 1.0 3.0 5.0 7.0 9.0 ... 351.0 353.0 355.0 357.0 359.0
  * time     (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2023-02-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

If the netCDF file names the latitude and longitude dimensions with other differently, we will also need to speicify them.
For instance, below we are trying to load the HadCRUT dataset, which have different names for the coordinates:

In [24]:
ds = xr.open_dataset(os.path.join(dirpath, 'HadCRUT.5.0.1.0.analysis.anomalies.ensemble_mean.nc'))
ds

<xarray.Dataset>
Dimensions:           (time: 2076, latitude: 36, longitude: 72, bnds: 2)
Coordinates:
  * time              (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-1...
  * latitude          (latitude) float64 -87.5 -82.5 -77.5 ... 77.5 82.5 87.5
  * longitude         (longitude) float64 -177.5 -172.5 -167.5 ... 172.5 177.5
    realization       int64 ...
Dimensions without coordinates: bnds
Data variables:
    tas_mean          (time, latitude, longitude) float64 ...
    time_bnds         (time, bnds) datetime64[ns] ...
    latitude_bnds     (latitude, bnds) float64 ...
    longitude_bnds    (longitude, bnds) float64 ...
    realization_bnds  (bnds) int64 ...
Attributes:
    comment:      2m air temperature over land blended with sea water tempera...
    history:      Data set built at: 2023-01-10T22:32:59+00:00
    institution:  Met Office Hadley Centre / Climatic Research Unit, Universi...
    licence:      HadCRUT5 is licensed under the Open Government Licence v3.0...
    reference:    C. P. Morice, J. J. Kennedy, N. A. Rayner, J. P. Winn, E. H...
    source:       CRUTEM.5.0.1.0 HadSST.4.0.0.0
    title:        HadCRUT.5.0.1.0 blended land air temperature and sea-surfac...
    version:      HadCRUT.5.0.1.0
    Conventions:  CF-1.7

We specify the `lon_name` and `lat_name` in the arguments so that the `.load_nc()` method can load the data correctly.
`time_name` may be also specified if its named differently in the netCDF file.
Once loaded, those coordinates will be renamed to the standard `time`, `lat`, and `lon`.

In [26]:
dirpath = './data'
fd = cfr.ClimateField().load_nc(
    os.path.join(dirpath, 'HadCRUT.5.0.1.0.analysis.anomalies.ensemble_mean.nc'),
    time_name='time',       # specify the name of the time dimension
    lon_name='longitude',   # specify the name of the lontitude dimension
    lat_name='latitude',    # specify the name of the latitude dimension
    vn='tas_mean',          # specify the name of the variable to load
)
fd.da  # check the loaded `xarray.DataArray`

<xarray.DataArray 'tas_mean' (time: 2076, lat: 36, lon: 72)>
[5380992 values with dtype=float64]
Coordinates:
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:...
  * lat          (lat) float64 -87.5 -82.5 -77.5 -72.5 ... 72.5 77.5 82.5 87.5
  * lon          (lon) float64 2.5 7.5 12.5 17.5 ... 342.5 347.5 352.5 357.5
    realization  int64 ...
Attributes:
    long_name:     blended air_temperature_anomaly over land with sea_water_t...
    units:         K
    cell_methods:  area: mean (interval: 5.0 degrees_north 5.0 degrees_east) ...

## `cfr.ClimateField` => a netCDF file

A `cfr.ClimateField` can be output to a netCDF file easily with the `.to_nc()` method:

In [28]:
fd.to_nc('./data/fd.nc')

ClimateField.da["tas_mean"] saved to: ./data/fd.nc


Now let's check if the saved netCDF file looks fine:

In [30]:
da = xr.open_dataarray('./data/fd.nc')
da

<xarray.DataArray 'tas_mean' (time: 2076, lat: 36, lon: 72)>
[5380992 values with dtype=float64]
Coordinates:
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:...
  * lat          (lat) float64 -87.5 -82.5 -77.5 -72.5 ... 72.5 77.5 82.5 87.5
  * lon          (lon) float64 2.5 7.5 12.5 17.5 ... 342.5 347.5 352.5 357.5
    realization  int64 ...
Attributes:
    long_name:     blended air_temperature_anomaly over land with sea_water_t...
    units:         K
    cell_methods:  area: mean (interval: 5.0 degrees_north 5.0 degrees_east) ...

## `xarray.DataArray` => `cfr.ClimateField`

Sometimes, we will load a `xarray.DataArray` first, after which we may convert it to a `cfr.ClimateField` with the `.from_da()` method:

In [31]:
fd = cfr.ClimateField().from_da(da)
fd.da

<xarray.DataArray 'tas_mean' (time: 2076, lat: 36, lon: 72)>
[5380992 values with dtype=float64]
Coordinates:
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:...
  * lat          (lat) float64 -87.5 -82.5 -77.5 -72.5 ... 72.5 77.5 82.5 87.5
  * lon          (lon) float64 2.5 7.5 12.5 17.5 ... 342.5 347.5 352.5 357.5
    realization  int64 ...
Attributes:
    long_name:     blended air_temperature_anomaly over land with sea_water_t...
    units:         K
    cell_methods:  area: mean (interval: 5.0 degrees_north 5.0 degrees_east) ...

## `cfr.ClimateField` => `xarray.DataArray`

The convertion from a `cfr.ClimateField` to a `xarray.DataArray` is trivial: simply access the `.da` attribute:

In [33]:
da = fd.da
da

<xarray.DataArray 'tas_mean' (time: 2076, lat: 36, lon: 72)>
[5380992 values with dtype=float64]
Coordinates:
  * time         (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:...
  * lat          (lat) float64 -87.5 -82.5 -77.5 -72.5 ... 72.5 77.5 82.5 87.5
  * lon          (lon) float64 2.5 7.5 12.5 17.5 ... 342.5 347.5 352.5 357.5
    realization  int64 ...
Attributes:
    long_name:     blended air_temperature_anomaly over land with sea_water_t...
    units:         K
    cell_methods:  area: mean (interval: 5.0 degrees_north 5.0 degrees_east) ...